# Surface Brightness Measurements with PS1 Images

Brief tutorial on measuring surface brightnesses from PS1!  The steps are just downloading the PS1 images, doing aperture photometry, and converting to surface brightness.  First, in Anaconda:

```conda install astropy
conda install photutils```

Then, you'll have to download the version of Panstamps I forked from thespacedoctor on github and converted to python 3: 

```git clone https://github.com/djones1040/panstamps.git
cd panstamps
python setup.py install```

Panstamps should work w/i Python, but I'm just gonna use os.system to do things at the command line.

In [1]:
# initial imports
import numpy as np
import pylab as plt
import os
from astropy.io import fits
from astropy.coordinates import SkyCoord
import astropy.units as u
from photutils import CircularAperture,SkyEllipticalAperture,aperture_photometry,EllipticalAnnulus
from photutils.utils import calc_total_error
from astropy.coordinates import SkyCoord
from astropy.stats import sigma_clipped_stats
from astropy import wcs
import glob
import cosmo

Then, choose a random SN - 2018gv is easy

In [2]:
snid = '2018gv'
ra = 121.3940833
dec = -11.4380194
z = 0.005274
snid = 'whirlpool'
ra = 202.48417
dec = 47.230560

## Downloading the Images

Run the Panstamps command - give this a couple min, seems like Jupyter isn't really reporting the full output but it's running.  It says it's done before it's actually done!

In [ ]:
# https://panstamps.readthedocs.io/en/latest/_includes/index.html#documentation
if not os.path.exists('ps1_images/%s'%snid):
    os.makedirs('ps1_images/%s'%snid)

# fits files, 60 arcmin width,custom download folder
panstamps_cmd = "panstamps -f --width 60 --downloadFolder ps1_images/%s stack %.7f %.7f"%(snid,ra,dec)
print("running '%s'"%panstamps_cmd)
os.system(panstamps_cmd)

running 'panstamps -f --width 60 --downloadFolder ps1_images/whirlpool stack 202.4841700 -11.4380194'


## Measuring Surface Brightness

Let's measure surface brightness w/i a 2 kpc radius.  A little big for 2018gv, but should be ok for most of the SNe farther out in the Hubble flow.

In [3]:
## grab the images, figure out where in each image the SN position is in each
files = glob.glob('ps1_images/%s/*fits'%snid)

# for each image, get obs filter and image coordinates for the SN
filters,xpos,ypos = [],[],[]
for f in files:
    print(f)
    filters += [fits.getval(f,'FPA.FILTER').split('.')[0]]
    ps1_wcs = wcs.WCS(fits.getheader(f))
    snx,sny = ps1_wcs.wcs_world2pix([(ra,dec)],0)[0]
    xpos += [snx]; ypos += [sny]

ps1_images/whirlpool/stack_r_ra202.484170_dec47.230560_arcsec1500_skycell2261.085.fits
ps1_images/whirlpool/stack_i_ra202.484170_dec47.230560_arcsec1500_skycell2261.085.fits
ps1_images/whirlpool/stack_g_ra202.484170_dec47.230560_arcsec1500_skycell2261.085.fits


this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]


In [10]:
apr_arcsec = 1.5 #(~1 FWHM)
# arcsec to number of PS1 pixels (0.25 arcsec/pix)
apr_pix = apr_arcsec/0.25

In [11]:
## routine for doing the aperture photometry
## note that no sky subtraction is done because PS1 images should already be sky-subtracted
def doAperPhot(xpos,ypos,image,exptime,zpt,aprad,errorim=[],unit='counts/sec'):

    if type(xpos) == np.ndarray:
        positions = [(x,y) for x,y in zip(xpos,ypos)]
    else:
        positions = [(xpos,ypos)]

    if not len(errorim):
        # get the uncertainty from the sky standard deviation, sigma-clipped STD
        avg,sky,skystd = sigma_clipped_stats(
            image[(image != 0)],
            sigma=3.0)

    ap = CircularAperture(positions,r=aprad)

    # this is a big weird, but part of checking if aperture extends beyond image
    apermask = ap.to_mask(method='exact')[0]
    imshape = np.shape(image)
    apermask_full = apermask.to_image(imshape)
    apermask_cols = np.where(apermask_full > 0)
    if apermask.bbox.ixmin < 0 or apermask.bbox.iymin < 0 or \
       apermask.bbox.ixmax > imshape[1]-1 or apermask.bbox.iymax > imshape[0]-1:
        print('Error : aperture extends outside the image!!')
        if type(xpos) == np.ndarray:
            return(np.array([np.nan]*len(xpos)),np.array([np.nan]*len(xpos)),np.array([np.nan]*len(xpos)),
                   np.array([np.nan]*len(xpos)),np.array([1]*len(xpos)))
        else:
            return(np.nan,np.nan,np.nan,np.nan,1)
    try:
        apOutsideImageCols = np.where((apermask_cols[0] == 0) | (apermask_cols[1] == 0) |
                                      (apermask_cols[0] == imshape[0]-1) | (apermask_cols[1] == imshape[1]-1))[0]
    except:
        import pdb; pdb.set_trace()

    # check if aperture extends outside the image
    if len(apOutsideImageCols):
        print('Error : aperture extends outside the image!!')
        if type(xpos) == np.ndarray:
            return(np.array([np.nan]*len(xpos)),np.array([np.nan]*len(xpos)),
                   np.array([sky]*len(xpos)),np.array([np.nan]*len(xpos)),np.array([1]*len(xpos)))
        else:
            return(np.nan,np.nan,sky,np.nan,1)

    if len(errorim):
        phot_table = aperture_photometry(image, ap, error=errorim)
    else:
        # calc error from sky standard deviation and image data (poisson noise & sky std)
        if unit == 'counts/sec':
            err = calc_total_error(image, skystd, exptime)
        else:
            err = calc_total_error(image, skystd, 1.0)
        phot_table = aperture_photometry(image, ap, error=err)

    if type(xpos) == np.ndarray:
        flux = phot_table['aperture_sum']	
        fluxerr = phot_table['aperture_sum_err']#**2.#+ skystd**2./ellap.area() )
    else:
        flux = phot_table['aperture_sum'][0]	
        fluxerr = phot_table['aperture_sum_err'][0]#**2.#+ skystd**2./ellap.area() )

    mag = -2.5*np.log10(flux) + zpt
    magerr = 2.5/np.log(10)*fluxerr/flux

    return(mag,magerr,flux,fluxerr,0)


In [12]:
## let's do some aperture photometry!
for f,x,y,flt in zip(files,xpos,ypos,filters):
    image = fits.getdata(f)
    header = fits.getheader(f)
    # PS1 zeropoints are set to 25 + 2.5*log10(exptime)
    mag,magerr,flux,fluxerr,badflag = \
        doAperPhot(x,y,image,header['EXPTIME'],25.+2.5*np.log10(float(header['EXPTIME'])),
                   apr_pix,unit='counts')
    # wikipedia says surface brightness is mag + 2.5*np.log10(area)
    sb = mag + 2.5*np.log10(np.pi*apr_arcsec**2.)
    print('%s: %.3f'%(flt,sb))

r: 22.183


i: 21.747


g: 23.150
